# MODEL - IMAGE LOADING & NEURAL NETWORK

In [1]:
#Import libraries
import os
import io
import cv2
from PIL import Image
import h5py
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras

2024-09-09 14:29:17.974948: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-09 14:29:17.989546: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-09 14:29:18.017571: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-09 14:29:18.050737: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-09 14:29:18.059654: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-09 14:29:18.074063: I tensorflow/core/platform/cpu_feature_gu

## 1) DATA LOADING - BASIC

In [2]:
#General file paths
projectDir = os.getcwd() + "/"
parentDir = os.path.abspath(os.path.join(projectDir, os.pardir)) + "/"
dataPath = os.path.abspath(os.path.join(projectDir, os.pardir)) + "/isic-2024-challenge/"

#Metadata file paths
metaPath = dataPath + "train-metadata.csv"

#Image file path
file = dataPath + "train-image.hdf5"

#Image subset: normal, hairs1, hairs2, wrinkles1, wrinkles2, protrusions, malignant, malignant
image_files = ["ISIC_0015670", "ISIC_0052213", "ISIC_0075726", "ISIC_0076172", "ISIC_8570031", "ISIC_5071401", "ISIC_0104229", "ISIC_9877311"]

In [3]:
#Import metadata
metadata = pd.read_csv(metaPath, sep=",")

#Import images from hdf5 file and one image
images = []
f = h5py.File(file, mode="r")
for isic_id in image_files:
    image = np.array(
        Image.open(
            io.BytesIO(f[isic_id][()])
            )
        )
    images.append(image)

/tmp/ipykernel_95855/2560820901.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(metaPath, sep=",")


## 2) GENERAL FUNCTIONS

In [4]:
#Function to show image
def show_img(image):
    plt.imshow(image, interpolation=None)
    plt.grid(None)
    plt.show()

In [5]:
#Image cropping
def crop_image(images_list, nbPix = 100):
    output_images = []
    for image in images_list:
        #Height adjustments
        h = len(image)
        adj = len(image) - nbPix
        h1 = round(adj / 2) #Top
        h2 = h - (adj - h1) #Bottom

        #Width adjustments
        w = len(image[0])
        w_adj = w - nbPix
        w1 = round(w_adj / 2) #Left
        w2 = w - (w_adj - w1) #Right

        img = image[h1:h2,w1:w2]
        output_images.append(img)
        
    return np.array(output_images)

In [23]:
#IMAGES: cropped to 100 pixels and cast to tensorflow compatible float32
X_img = [tf.cast(img, tf.float32) for img in crop_image(images, nbPix=100)]

#METADATA: color and size features having no NAs
X_meta = metadata[["isic_id",
                   "clin_size_long_diam_mm",
                   "tbp_lv_areaMM2",
                   "tbp_lv_area_perim_ratio",
                   "tbp_lv_eccentricity",
                   "tbp_lv_minorAxisMM",
                   "tbp_lv_color_std_mean",
                   "tbp_lv_deltaLBnorm",
                   "tbp_lv_radial_color_std_max"]]

X_meta = X_meta[X_meta["isic_id"].isin(image_files)].iloc[:,1:]

#TARGET
y = metadata[metadata["isic_id"].isin(image_files)]["target"]

#Verify that there are no NAs
print("-- X_meta NA counts --")
print(X_meta.isna().sum())
print("\n-- y NA count --")
print(y.isna().sum())

-- X_meta NA counts --
clin_size_long_diam_mm         0
tbp_lv_areaMM2                 0
tbp_lv_area_perim_ratio        0
tbp_lv_eccentricity            0
tbp_lv_minorAxisMM             0
tbp_lv_color_std_mean          0
tbp_lv_deltaLBnorm             0
tbp_lv_radial_color_std_max    0
dtype: int64

-- y NA count --
0


In [34]:
#Generates the image (standardized). Avoids multiple file open/read/close operations.
#file: full path for file
#imgs: list of images to load
#imgSize: number of pixels for size/resolution adjustment in square form
class hdf5_generator:
    def __init__(self, file, imgs, imgSize):
        self.file = file
        self.imgs = imgs
        self.imgSize = imgSize
        self.f = h5py.File(file, mode="r")
    def __call__(self):
        with h5py.File(self.file, 'r') as h5file:
            for img in self.imgs:
                img = np.array(Image.open(io.BytesIO(f[img][()])))
                img = tf.image.resize(img, [self.imgSize, self.imgSize])
                img = tf.constant(np.reshape(img/255,(1,100,100,3)), dtype=tf.float32) #standardized here
                yield img/255

imgSize = 100
features_dataset = tf.data.Dataset.from_generator(
    hdf5_generator(file, image_files, imgSize),
    output_types=tf.float32,
    output_shapes = tf.TensorShape([1, imgSize,imgSize,3])
    )

#Generate target dataset
y = [np.reshape(element, (1,1)) for element in y]
y = tf.cast(y, dtype=tf.int32)
labels_dataset = tf.data.Dataset.from_tensor_slices(y, name = "target")

#Generate metadata set
X_meta = tf.cast(X_meta, dtype=tf.int32)
meta_dataset = tf.data.Dataset.from_tensor_slices(X_meta, name = "metadata")

#Combine datasets into one
dataset = tf.data.Dataset.zip((features_dataset, meta_dataset))
dataset = tf.data.Dataset.zip((dataset, labels_dataset))

In [33]:
#Examine the dataset objects
iterator = iter(dataset)
print(meta_dataset)
print(iterator.get_next())

<_TensorSliceDataset element_spec=TensorSpec(shape=(8,), dtype=tf.int32, name=None)>
(<tf.Tensor: shape=(1, 100, 100, 3), dtype=float32, numpy=
array([[[[0.00277958, 0.0022567 , 0.00207216],
         [0.00277716, 0.00225429, 0.00206974],
         [0.00282215, 0.00226928, 0.00208474],
         ...,
         [0.00300338, 0.002389  , 0.00226559],
         [0.00291981, 0.00234181, 0.0022034 ],
         [0.00290116, 0.00233215, 0.00219374]],

        [[0.00287166, 0.00234879, 0.00216424],
         [0.00286455, 0.00233115, 0.0021466 ],
         [0.00288695, 0.00233364, 0.00214909],
         ...,
         [0.00290403, 0.00230719, 0.002175  ],
         [0.00285521, 0.00227347, 0.00213506],
         [0.00283383, 0.00226307, 0.00212466]],

        [[0.00297855, 0.00242568, 0.00224114],
         [0.00295762, 0.00240431, 0.00221976],
         [0.00295156, 0.00239792, 0.00221338],
         ...,
         [0.00278582, 0.00220143, 0.00206302],
         [0.00279953, 0.00221514, 0.00207673],
         [0

## 4) CNN MODEL

In [40]:
#Simple CNN model using only images and target
class CNN_model(tf.keras.Model):
    def __init__(self, neurons = 8, activ = 'tanh'):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=16, kernel_size=5, strides=(1, 1), activation='relu', padding='same', input_shape=(100, 100, 3))
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=(2,2))
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(neurons, activation = activ)
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x_image, x_meta = inputs

        # Convolutions
        x1 = self.conv1(x_image)
        x1 = self.pool1(x1)

        # Flattening of images for input layer
        x1 = self.flatten(x1)

        # Hidden layers of neural network
        x1 = self.dense1(x1)

        # Output layer of neural network
        output = self.dense2(x1)

        return output

#Hybrid CNN model taking metadata
class Hybrid_model(tf.keras.Model):
    def __init__(self, neurons = 8, activ = 'tanh'):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=16, kernel_size=5, strides=(1, 1), activation='relu', padding='same')
        self.conv2 = tf.keras.layers.Conv2D(32, 5, activation='relu')
        self.pool = tf.keras.layers.MaxPool2D(pool_size=(2,2))
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(neurons, activation = activ)
        self.dense2 = tf.keras.layers.Dense(neurons, activation = activ)
        self.dense3 = tf.keras.layers.Dense(1, activation='sigmoid')
        #self.dropout = tf.keras.layers.dropout(0.25)

    def call(self, inputs, training=False):
        x_image, x_meta = inputs
        # Convolutions
        x1 = self.conv1(x_image)
        x1 = self.pool(x1)
        #x1 = self.conv2(x1)
        #x1 = self.pool(x1)
        # Flattening of images and concatenation with other data
        x1 = self.flatten(x1)
        x_all = tf.keras.concatenate([x1,x_meta])
        # Neural Network
        x1 = self.dense1(x_all)
        #x = self.dense2(x_all)
        #if training:
        #    x_all = self.dropout(x_all, training=training)
        output = self.dense3(x_all)
        return output

In [41]:
#Set seed
tf.random.set_seed(71)

#Initialize model
model = CNN_model(neurons=8, activ='tanh')

#Define optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True,
                                          label_smoothing=0.0,
                                          axis=-1,
                                          reduction='sum_over_batch_size',
                                          name='binary_crossentropy')

#Compile the model with loss, optimizer, and metrics
model.compile(loss = loss,
              optimizer = optimizer,
              metrics = [
                  tf.keras.metrics.BinaryAccuracy(),
                  tf.keras.metrics.FalseNegatives(),
                  tf.keras.metrics.FalsePositives(),
                  tf.keras.metrics.TrueNegatives(),
                  tf.keras.metrics.TruePositives()
                  ]
)

## 3) DATA PREPARATION

In [42]:
#Fit the model
mod = model.fit(dataset, epochs=4)

Epoch 1/4
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - binary_accuracy: 0.5642 - false_negatives_5: 0.7778 - false_positives_4: 1.0000 - loss: 0.6600 - true_negatives_4: 3.1111 - true_positives_4: 0.0000e+00
Epoch 2/4
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - binary_accuracy: 0.8786 - false_negatives_5: 0.7778 - false_positives_4: 0.0000e+00 - loss: 0.4049 - true_negatives_4: 4.1111 - true_positives_4: 0.0000e+00   
Epoch 3/4
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - binary_accuracy: 0.8786 - false_negatives_5: 0.7778 - false_positives_4: 0.0000e+00 - loss: 0.3794 - true_negatives_4: 4.1111 - true_positives_4: 0.0000e+00  
Epoch 4/4
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - binary_accuracy: 0.8786 - false_negatives_5: 0.7778 - false_positives_4: 0.0000e+00 - loss: 0.3582 - true_negatives_4: 4.1111 - true_positives_4: 0.0000e+00    


2024-09-09 14:50:55.027012: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [22]:
mod.history

{'binary_accuracy': [0.6666666865348816,
  0.6666666865348816,
  0.6666666865348816,
  0.6666666865348816],
 'false_negatives_1': [2.0, 2.0, 2.0, 2.0],
 'false_positives': [0.0, 0.0, 0.0, 0.0],
 'loss': [0.5017471313476562,
  0.5017471313476562,
  0.5017470717430115,
  0.5017469525337219],
 'true_negatives': [6.0, 6.0, 6.0, 6.0],
 'true_positives': [0.0, 0.0, 0.0, 0.0]}